In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None, max_feat = 10):
    
    #print("Actual value ", test_y)
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=max_feat, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = None):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        
        #if shap_type == "kernel":
        #    shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)
        #elif shap_type == "tree":
        #    shap_explainer = shap.TreeExplainer(cls)
        #elif shap_type == "deep":
        #    shap_explainer = shap.DeepExplainer(cls, background)
        
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        if top != None:
            for i in range(top):
                feat = importances[i]
                if feat[2] > 0:
                    rel_feat.append(feat)

            rel_exp.append(rel_feat)
        else:
            rel_exp = exp
        
    return exp, rel_exp

In [8]:
def dispersal(weights, features):
    feat_len = len(features)
    #print(feat_len)
    weights_by_feat = []
       
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
        
    #for iteration in weights:
     #   for val in iteration:
      #      idx = iteration.index(val)
       #     print(idx)
        #    weights_by_feat[idx].append(val)
    
    dispersal = []
    dispersal_no_outlier = []
    
    for each in weights_by_feat:
        #print("Feature", weights_by_feat.index(each)+1)
        mean = np.mean(each)
        std_dev = np.std(each)
        var = std_dev**2
        
        if mean == 0:
            dispersal.append(0)
            dispersal_no_outlier.append(0)
        #print(each)
        else:
            #dispersal with outliers
            rel_var = var/abs(mean)
            dispersal.append(rel_var)
            
            #dispersal without outliers - remove anything with a z-score higher
            #than 3 (more than 3 standard deviations away from the mean)
            rem_outlier = []
            z_scores = stats.zscore(each)
            #print(z_scores)
            #print("New list:")
            for i in range(len(z_scores)):
                #print(each[i],":",z_scores[i])
                if -3 < z_scores[i] < 3:
                    rem_outlier.append(each[i])
                #print(rem_outlier)
            if rem_outlier != []:
                new_mean = np.mean(rem_outlier)
                if new_mean == 0:
                    dispersal_no_outlier.append(0)
                else:
                    new_std = np.std(rem_outlier)
                    new_var = new_std**2
                    new_rel_var = new_var/abs(new_mean)
                    dispersal_no_outlier.append(new_rel_var)
            else:
                dispersal_no_outlier.append(rel_var)
    #print(dispersal_no_outlier)
    return dispersal, dispersal_no_outlier

In [9]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = True
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10
max_prefix = 25

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [10]:
if generate_samples:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))])
            
            for bucket in range(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

                predictor = joblib.load(pipeline_path)
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                bucketer = joblib.load(bucketer_path)

                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previous results from predictions
                tn_path = os.path.join(PATH, "%s/%s_%s/instances/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/instances/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/instances/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/instances/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                instances.append(tn_list)
                instances.append(tp_list)
                instances.append(fn_list)
                instances.append(fp_list)

                #choose instances from the event log to explain, based on different prefix lengths
                sample_instances = []
                
                for each in instances:
                    prefixes = range(0, max_prefix+1, gap)
                    
                    sample = []
                    for length in prefixes:
                        #print(length)
                        #Find instances of relevant length
                        relevant = [d for d in each if (d['nr_events'] == length)]
                        #Find instances of different prediction probabilities
                        prs = [0.5, 0.6, 0.7, 0.8, 0.9, 1.1]
                        for i in list(range(len(prs)-1)):
                            low = prs[i]
                            high = prs[i+1]
                            ins = [d for d in relevant if (d['proba'] >= low) & (d['proba'] < high)]
                            if len(ins) >= sample_size:
                                sample.extend(random.sample(ins, k=sample_size))
                            else:
                                sample.extend(ins)
                    sample = sorted(sample, key = lambda i: (i['proba'], i['nr_events']))
                    sample_instances.append(sample)

                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1


In [11]:
if generate_model_shap:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))])
            
            for bucket in range(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                #X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                #Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                #with open (X_train_path, 'rb') as f:
                #    dt_train_bucket = pickle.load(f)
                #with open (Y_train_path, 'rb') as f:
                #    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #create explainers now that can be passed later
                tree_explainer = shap.TreeExplainer(cls)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    
                    for instance in category:
                        instance_no += 1    
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                
                        group = instance['input']

                        #print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
    
                        #print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get Tree SHAP explanations for instance
                        exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x_group, feat_list, top = max_feat)

                        feat_pres = []
                        feat_weights = []

                        for iteration in rel_exp:
                            print("Computing feature presence for iteration", rel_exp.index(iteration))
                            
                            presence_list = [0]*len(feat_list)
                           
                            for each in feat_list:
                                list_idx = feat_list.index(each)

                                for explanation in iteration:
                                    if each in explanation[0]:
                                        presence_list[list_idx] = 1

                            feat_pres.append(presence_list)
                            
                        for iteration in exp:
                            print("Compiling feature weights for iteration", exp.index(iteration))
                            
                            weights = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        
                                        weights[list_idx] = explanation[1]
                            feat_weights.append(weights)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['tree_shap_stability'] = stability
                        
                        rel_var, second_var = dispersal(feat_weights, feat_list)
                        avg_dispersal = np.mean(rel_var)
                        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
                        instance['shap_weights_dispersal'] = rel_var
                        adj_dispersal = np.mean(second_var)
                        print ("Dispersal with no outliers:", round(adj_dispersal, 2))
                        instance['adjusted_shap_weights_dispersal'] = second_var
                        
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 250 .
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 1 of 4 . Te

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 1 of 4 . Testing 11 of 250 .
Computing feature presence f

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 1 of 4 . Testing 20 of 250 .
Computing feature presence f

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 1 of 4 . Testing 29 of 250 .
Computing feature presence f

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 1 of 4 . Testing 38 of 250 .
Computing feature presence f

...

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 4 of 4 . Testing 215 of 250 .
Computing feature presence 

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 4 of 4 . Testing 224 of 250 .
Computing feature presence 

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 4 of 4 . Testing 233 of 250 .
Computing feature presence 

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0
Category 4 of 4 . Testing 242 of 250 .
Computing feature presence 

Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Computing feature presence for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Compiling feature weights for iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Dispersal with no outliers: 0.0


In [12]:
if generate_lime:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))])
            
            for bucket in range(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #get the training data as a matrix
                trainingdata = feature_combiner.fit_transform(dt_train_bucket);
                
                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    for instance in category:
                        instance_no += 1
                        
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                        
                        group = instance['input']
                        
                        #create explainer now that can be passed later
                        class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
                        lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                          feature_names = feature_combiner.get_feature_names(),
                                          class_names=class_names, discretize_continuous=True)

                        #print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        #print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get lime explanations for instance
                        feat_pres = []
                        feat_weights = []

                        for iteration in list(range(exp_iter)):
                            print("Run", iteration)
                            
                            lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, instance['actual'], max_feat = len(feat_list))

                            presence_list = [0]*len(feat_list)
                            weights = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in lime_exp.as_list():
                                    if each in explanation[0]:
                                        if lime_exp.as_list().index(explanation) < max_feat:
                                            presence_list[list_idx] = 1
                                        weights[list_idx] = explanation[1]

                            feat_pres.append(presence_list)
                            feat_weights.append(weights)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['lime_stability'] = stability
                        
                        rel_var, second_var = dispersal(feat_weights, feat_list)
                        avg_dispersal = np.mean(rel_var)
                        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
                        instance['lime_weights_dispersal'] = rel_var
                        adj_dispersal = np.mean(second_var)
                        print ("Dispersal with no outliers:", round(adj_dispersal, 2))
                        instance['adjusted_lime_weights_dispersal'] = second_var
                                        
                #Save dictionaries updated with stability scores
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 1 of 4 . Testing 2 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 3 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.61
Dispersal with no outliers: 0.61
Category 1 of 4 . Testing 4 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 1 of 4 . Testing 5 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.87
Dispersal of feature importance: 0.1
Dispersal with no outliers: 0.1
Category 1 of 4 . Testing 6 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5


Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 1 of 4 . Testing 46 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.86
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 1 of 4 . Testing 47 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.17
Dispersal with no outliers: 0.17
Category 1 of 4 . Testing 48 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 4.24
Dispersal with no outliers: 4.24
Category 1 of 4 . Testing 49 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 50 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importan

Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 1 of 4 . Testing 91 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.16
Dispersal with no outliers: 0.16
Category 1 of 4 . Testing 92 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.84
Dispersal of feature importance: 0.14
Dispersal with no outliers: 0.14
Category 1 of 4 . Testing 93 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 1 of 4 . Testing 94 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.84
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 95 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82


Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 135 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 1 of 4 . Testing 136 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.16
Dispersal with no outliers: 0.16
Category 1 of 4 . Testing 137 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.53
Dispersal with no outliers: 0.53
Category 1 of 4 . Testing 138 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 139 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importanc

Stability: 0.79
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 1 of 4 . Testing 179 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 1 of 4 . Testing 180 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 1 of 4 . Testing 181 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.03
Dispersal with no outliers: 0.03
Category 1 of 4 . Testing 182 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.2
Dispersal with no outliers: 0.2
Category 1 of 4 . Testing 183 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.1
Dispersal with no ou

...

Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 52 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.09
Dispersal with no outliers: 0.09
Category 4 of 4 . Testing 53 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 54 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 55 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.86
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 56 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83


Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 97 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 98 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.85
Dispersal of feature importance: 0.49
Dispersal with no outliers: 0.49
Category 4 of 4 . Testing 99 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.84
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 100 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 4 of 4 . Testing 101 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.

Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 4 of 4 . Testing 141 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 142 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.03
Dispersal with no outliers: 0.03
Category 4 of 4 . Testing 143 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 144 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 145 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.83
Dispersal of featur

Run 7
Run 8
Run 9
Stability: 0.85
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 4 of 4 . Testing 185 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.85
Dispersal of feature importance: 0.05
Dispersal with no outliers: 0.05
Category 4 of 4 . Testing 186 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 4 of 4 . Testing 187 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.08
Dispersal with no outliers: 0.08
Category 4 of 4 . Testing 188 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.07
Dispersal with no outliers: 0.07
Category 4 of 4 . Testing 189 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.05

Stability: 0.85
Dispersal of feature importance: 0.04
Dispersal with no outliers: 0.04
Category 4 of 4 . Testing 229 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.68
Dispersal with no outliers: 0.68
Category 4 of 4 . Testing 230 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.89
Dispersal with no outliers: 0.89
Category 4 of 4 . Testing 231 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.84
Dispersal of feature importance: 0.17
Dispersal with no outliers: 0.17
Category 4 of 4 . Testing 232 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.09
Dispersal with no outliers: 0.09
Category 4 of 4 . Testing 233 of 250 .
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.05
Dispersal with n